In [1]:
import java.io.File
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.hive.HiveContext

val hiveContext = new HiveContext(sc)
val warehouseLocation = "hdfs://namenode:8020/user/hive/warehouse"
val data_base = "db_nubank"
var table = ""
var dir_path = warehouseLocation + "/" + data_base + ".db/" + table
var db_table = data_base + "." + table

spark.sql("use db_nubank")

Intitializing Scala interpreter ...

Spark Web UI available at http://8ea9f1e3afef:4040
SparkContext available as 'sc' (version = 3.0.2, master = local[*], app id = local-1615748011742)
SparkSession available as 'spark'


import java.io.File
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.hive.HiveContext
hiveContext: org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@209d5f51
warehouseLocation: String = hdfs://namenode:8020/user/hive/warehouse
data_base: String = db_nubank
table: String = ""
dir_path: String = hdfs://namenode:8020/user/hive/warehouse/db_nubank.db/
db_table: String = db_nubank.
res0: org.apache.spark.sql.DataFrame = []


In [38]:
val df_transfer_union = spark.read.format("csv").option("header", "true").load("file:///dataset/nubank/out_transfer_union/part-00000-5bfcdad5-e389-4763-abd1-4439eb7b3d71-c000.csv")
df_transfer_union.createOrReplaceTempView("tmp_transfer_union")
df_transfer_union.printSchema()

// val df_transfer_union_distinct = df_transfer_union.select("account_id","action_month","action_year").where($"flag_is_pix" === 1).distinct()
val df_transfer_union_distinct = df_transfer_union.select("account_id","action_month","action_year","flag_is_pix").distinct()
df_transfer_union_distinct.createOrReplaceTempView("tmp_transfer_union_distinct")
df_transfer_union_distinct.printSchema()
spark.sql("""select t1.action_year,t1.action_month,count(distinct account_id)  
from tmp_transfer_union_distinct t1 
group by t1.action_year,t1.action_month
order by cast(t1.action_year as int),cast(t1.action_month as int)"""
         ).show()

root
 |-- id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- action_timestamp: string (nullable = true)
 |-- action_month: string (nullable = true)
 |-- action_year: string (nullable = true)
 |-- flag_is_pix: string (nullable = true)

root
 |-- account_id: string (nullable = true)
 |-- action_month: string (nullable = true)
 |-- action_year: string (nullable = true)
 |-- flag_is_pix: string (nullable = true)

+-----------+------------+--------------------------+
|action_year|action_month|count(DISTINCT account_id)|
+-----------+------------+--------------------------+
|       2020|           1|                      3793|
|       2020|           2|                      3790|
|       2020|           3|                      3804|
|       2020|           4|                      3803|
|       2020|           5|                      3804|
|       2020|           6|                      3790|


df_transfer_union: org.apache.spark.sql.DataFrame = [id: string, account_id: string ... 6 more fields]
df_transfer_union_distinct: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [account_id: string, action_month: string ... 2 more fields]


In [27]:
val sql_pix_client = """select t1.action_year,t1.action_month,count(1) number_new_pix_transfer
from tmp_transfer_union_distinct t1
where not exists (select 1 
                  from tmp_transfer_union_distinct t2
                  where t2.account_id = t1.account_id 
                  and t2.flag_is_pix = t1.flag_is_pix
                  and concat(cast(t2.action_year as string),lpad(cast(t2.action_month as string),2,'0'))  
                        < concat(cast(t1.action_year as string),lpad(cast(t1.action_month as string),2,'0')) 
                  )
and flag_is_pix = 1                  
group by t1.action_year,t1.action_month
order by cast(t1.action_year as int),cast(t1.action_month as int)"""

val sql_no_pix_client = """select t1.action_year,t1.action_month,count(1) number_no_pix_transfer
from tmp_transfer_union_distinct t1
where not exists (select 1 
                  from tmp_transfer_union_distinct t2
                  where t2.account_id = t1.account_id 
                  and t2.flag_is_pix = 1
                  and concat(cast(t2.action_year as string),lpad(cast(t2.action_month as string),2,'0'))  
                        <= concat(cast(t1.action_year as string),lpad(cast(t1.action_month as string),2,'0')) 
                  )
and flag_is_pix = 0
group by t1.action_year,t1.action_month
order by cast(t1.action_year as int),cast(t1.action_month as int)"""

val sql_pix_client_geral = """select 
t1.*, t2. number_no_pix_transfer
from tmp_pix_client t1
inner join tmp_no_pix_client t2
on t1.action_year = t2.action_year
and t1.action_month = t2.action_month
order by cast(t1.action_year as int),cast(t1.action_month as int)"""


val df_pix_client = spark.sql(sql_pix_client)
val df_no_pix_client = spark.sql(sql_no_pix_client)
df_pix_client.createOrReplaceTempView("tmp_pix_client")
df_no_pix_client.createOrReplaceTempView("tmp_no_pix_client")

val df_pix_client_geral = spark.sql(sql_pix_client_geral)

df_pix_client.show(false)
df_no_pix_client.show(false)
df_pix_client_geral.show(false)

///////////////////////////////////////////////////////////////////////////////////////////////////


val sql_number_trans = """select action_year
,action_month
,flag_is_pix
,count(1) number_trans 
from  transfer_union
group by action_year,action_month,flag_is_pix
order by action_year,action_month"""

val sql_value_trans = """select action_year
,action_month
,flag_is_pix
,cast(round(sum(case when amount > 0 then amount else 0 end),2) as decimal(12,2)) value_trans_in
,cast(round(sum(case when amount < 0 then amount else 0 end),2) as decimal(12,2)) value_trans_out 
from  transfer_union
group by action_year,action_month,flag_is_pix
order by action_year,action_month"""

val df_number_trans = spark.sql(sql_number_trans)
df_number_trans.printSchema()
val df_value_trans = spark.sql(sql_value_trans)
df_value_trans.printSchema()

df_number_trans.show(false)
df_value_trans.show(false)

+-----------+------------+-----------------------+
|action_year|action_month|number_new_pix_transfer|
+-----------+------------+-----------------------+
|2020       |1           |3604                   |
|2020       |2           |191                    |
|2020       |3           |78                     |
|2020       |4           |33                     |
|2020       |5           |31                     |
|2020       |6           |13                     |
|2020       |7           |8                      |
|2020       |8           |7                      |
|2020       |9           |6                      |
|2020       |10          |4                      |
|2020       |11          |1                      |
|2020       |12          |2                      |
+-----------+------------+-----------------------+

+-----------+------------+----------------------+
|action_year|action_month|number_no_pix_transfer|
+-----------+------------+----------------------+
|2020       |1           |189    

sql_pix_client: String =
select t1.action_year,t1.action_month,count(1) number_new_pix_transfer
from tmp_transfer_union_distinct t1
where not exists (select 1
                  from tmp_transfer_union_distinct t2
                  where t2.account_id = t1.account_id
                  and t2.flag_is_pix = t1.flag_is_pix
                  and concat(cast(t2.action_year as string),lpad(cast(t2.action_month as string),2,'0'))
                        < concat(cast(t1.action_year as string),lpad(cast(t1.action_month as string),2,'0'))
                  )
and flag_is_pix = 1
group by t1.action_year,t1.action_month
order by cast(t1.action_year as int),cast(t1.action_month as int)
sql_no_pix_client: String =
select t1.action_year,t1.action_month,count(1) number_no_pix_transfer
from tmp_transfer_...
